#  Leetcodeの復習

あえて言語化にこだわる。
とりあえず、新しく解いた問題を上に書き足していくが、もし関連する過去に解いた問があれば、それを浮上させてくる。



## 

## 284. [Peeking Iterator](https://leetcode.com/problems/peeking-iterator/)


Iteratorをwrapし、peekメソッドを持つ以下のようなiteratorクラスを実装せよ。

```
PeekingIterator(int[] nums) Initializes the object with the given integer array nums.
# int next() Returns the next element in the array and moves the pointer to the next element.
# bool hasNext() Returns true if there are still elements in the array.
# int peek() Returns the next element in the array without moving the pointer
```


### 解法

簡単。next_valというフィールドを持っておくだけ。初期化、nextの呼び出し後には必ずnext_valを必ず埋めておく(つまり、常に一つ先の要素を取り出しておく)、という実装をすることで、めっちゃシンプルに実装できるようになる。

```py
class PeekingIterator:
    def __init__(self, iterator):
        self._next = iterator.next()
        self._iterator = iterator

    def peek(self):
        return self._next

    def next(self):
        ret = self._next
        self._next = self._iterator.next() if self._iterator.hasNext() else None
        return ret
```


### 所感

`常に一つ先の要素を取り出しておく`という発想は思いつかなかった。脱帽。

## 341. [Flatten Nested List Iterator](https://leetcode.com/problems/flatten-nested-list-iterator/)

```
class NestedInteger:
#    def isInteger(self) -> bool:
#    def getInteger(self) -> int:
#    def getList(self) -> [NestedInteger]:   ##### 空配列を返すこともある！！！
```

上記のようなNestedIntegerの配列が渡される時、これをflattenして、順々に値を返すiteratorオブジェクトを実装せよ。実装するべきiteratorのinterfaceは以下。

```
class NestedIterator:
# NestedIterator(List<NestedInteger> nestedList) Initializes the iterator with the nested list nestedList.
# int next() Returns the next integer in the nested list.
# boolean hasNext() Returns true if there are still some integers in the nested list and false otherwise.
```

### 解法1. 初期化時にrecursionでtraverseしてlistを初期化 (思いついた)

まあ、これが一番に思いつく。ただ、iterator patternは、pointerだけを保持していいのでspaceをセーブできるというのが売りなので、そういう意味でこれを実装したら「僕デザインパターン知りません！」と堂々と言い放っているようなものなので注意。

あと、最初実装した時にわざわざrecursive funcからリストを返し、それを呼び出しがわでextendしていくという謎実装をしていた。そんなことしない、かくfunc内で(global変数あるいは)インスタンス変数のリストにappendしていく。


### 解法2. 初期化時にstackを使用してtraverseしてlistを初期化(思いついた)

解法1のrecursionの代わりにstackを用いただけ。最初にNestedIntegerの配列をリストに逆順につめ、これをstackとして用いる。
これも内部でまるっとデータ構造持っておかないといけないのでいい解法とは言えない。



### 解法3. stackを利用、#next()が呼ばれる度に次の要素までtraverse

解法2とアイデアは同じ。解法1のrecursionの代わりにstackを用いただけ。最初にNestedIntegerの配列をリストに逆順につめ、これをstackとして用いる。NestedIntegerはポインタでしかないので、spaceをsaveできる。ただし、与えられたリストの要素が全てintを表すNestedIntegerなら、解法2とspace complexityは同じという認識。


引っかかった点として、NestInteger.getList()は空配列になりうるというところ。したがって、`hasNext(), next()`が呼ばれた時点でtailの要素がlistのNestedIntegerだった場合、それを分解してやる(処理を繰り返す)必要がある。しかも、このロジックはhasNext()とnext()の両方で使用されるので、`def make_stack_top_an_integer(self)`としてメソッドに切り出すのが吉。

```
class NestedIterator:
    def __init__(self, nestedList: [NestedInteger]):
        self.stack = list(reversed(nestedList))
    
    def next(self) -> int:
        self.make_stack_top_an_integer()
        return self.stack.pop().getInteger()
    
    def hasNext(self) -> bool:
        self.make_stack_top_an_integer()
        return len(self.stack) > 0
```

### 解法4. Using a Generator

読み流した。yieldとかのお話。



### 所感

あとこの問題、「もう残りの要素がない状態で#next()が呼ばれたらどうしましょう？？？」って聞くと「こいつ優秀！」って思われるらしい。

解くこと自体は簡単。計算時間求めるのはややこい。
解答後、iterator patternについてまとめた。
https://github.com/kudojp/RoadToSuperCoder/blob/master/design_patterns.ipynb

振り返ってみるとかなり良問な気がしてきた。

## 117. [Populating Next Right Pointers in Each Node II](https://leetcode.com/problems/populating-next-right-pointers-in-each-node-ii/)

binary searchが与えられる。各nodeのnextフィールドに、その右隣のnodeを指すpointerを代入せよ。O(1)spaceで。


### 解法1 O(N) space

まあ、普通にqに同階層のnodeを詰めるBFSで実装する。


### 解法2 待望のO(1) space

BFS。現在traverseしているより一段下のlevelのnodesのnextフィールドを設定することで、qを使用しなくてもその階層の次のnodeをたどっていける。(一段下というのが肝)

```py

## 最初にごn

while leftmost is not None:
    curr = leftmost
    prev = None
    leftmost = None
    while curr is not None:
        for child in [curr.left, curr.right]:
            if child is None:
                continue
            if prev is None:    # when it is a first node in the level
                leftmost = child
                prev = child
                continue
            prev.next = child
            prev = prev.next
        curr = curr.next
```


### 所感

本当にびっくりするくらい美しい良問だと思う。脱帽。




## 532. [K-diff Pairs in an Array](https://leetcode.com/problems/k-diff-pairs-in-an-array/)

int[]が与えられる時、差がkとなるような数字のユニークな組み合わせは何通りあるか？

### O(N)時間: counterのhashを作成後にkeyをループする

```py
val_to_cnt = Counters(nums)
checked = set()
cnt = 0

for val in val_to_cnt:
    if val in checked:
        continue
    if val + k in val_cnt:  # +kの場合のみ考えることでsimplifyした。
        cnt += 1
return cnt
```

上では書いてないが、k = 0の場合が見事なエッジケースw。

### O(NlogN)時間: ソートしてから2pointers

実装はややこい。書き終わってみたらまあなるほどって感じ。
気が向いたらかく　TODO

### 所感

解法２は、spaceは解法１と同じO(N)、timeは解法１より遅い上に、解法１より実装がややこしいので、まあ、解法１を使うでしょ。



[参考] ソートのtimecomplexityは `O(logN)` ~ `O(N)`かかる。
(meege sortがO(N), quick sortがO(logN)~O(N))

- https://twitter.com/kdjp20/status/1389091424063213568
- https://twitter.com/kdjp20/status/1389817538062680066


## 1648. [Sell Diminishing-Valued Colored Balls](https://leetcode.com/problems/sell-diminishing-valued-colored-balls/) (挫折)

## 1539. [Kth Missing Positive Number](https://leetcode.com/problems/kth-missing-positive-number/)


strictly increasing orderのintのlistが与えられる。skipされた数字のうちk番目のものを求めよ。(e.g. [1,2,4,6], k=2 -> 5)

### 解法1 (できた)

前からループを回す。計算時間O(N)

### 解法２ (できた)

binary searchを使う。
配列のあるindexまでにどれだけ数字がskipされたかは、arr[idx] - idx で求められることを利用。


bisect_left的実装でidxを見つけた後、そのidxからbackwardに戻っていくのではなく、idx - 1からforwardに探索してやるとより直感的。

### 所感



いい binary searchの練習になった。解法2で、一つidxを戻してforwardにたどっていくのわかりやすいなーって思った。

## 875. [Koko Eating Bananas](https://leetcode.com/problems/koko-eating-bananas/)

複数のバナナの山(`int[]`)をh時間内に食べ切ることのできる最も遅いスピードを求めよ。ただし、同じ時間フレーム(1時間)以内に複数のバナナに手をつけることはできない。

### 解法1 (heapを使用、思いついた -> tle) 

計算時間がO(h logN)かかってしまいTLE。    
TODO: リンク貼る

### 解法2 (binary searchを使用し、O(N * log max_pile_size)

計算時間を`O(N * log max_pile_size)`まで縮められた。   
(bisect_leftの実装にミスって116/118でTLEを連発してしまった。Javaで書き直したが、そういう問題ではなかったw)    


```python    
low = math.ceil(sum(piles) / h)
high = max(piles)

# イメージは、[f,f,f,f,t,t,t,t,t](x)の中で、一番左のtのindexを取得する。
# これが一番の肝。helper functionでtaking_timeなんかを取ってhと比較したりするとややこしい。

while low < high:
    mid = (low + high) // 2
    is_possible = is_possible(mid)
    
    if is_possible: # speed must be slower than mid
        high = mid
    else: # h <= t
        low = mid + 1 # speed is higher or 
    return low     # low != len(piles)
```
        
        
### 所感

解法2はグッチさんとマックで1時間くらい考えてやっと思いついた。
helper_functionをうまく設定できるかが鍵だと思う。    
bisect_leftに関してまとめた。 https://twitter.com/kdjp20/status/1424250217268871170

## 300.  [Longest Increasing Subsequence](https://leetcode.com/problems/longest-increasing-subsequence/)

intの配列において、最長のLISの配列の長さを求める問題。O(NlogN)時間でとけ。
(ex) [0,3,1,6,2,2,7] → [0,1,2,7] なので、ans = 4

### 解法1 (似たようなの思いついた)

与えられた配列の各要素に対応し、その要素までのLISの配列長を表すリストdpを作成していく。
初期条件は、dp[0] = 1。
dp[1:]に関しては、max(それまでに登場した要素で、現在の要素の値未満の要素のLIS長)+1。なお、そこまでにその値未満の要素が存在しない場合には1を入れる。

### 解法2 (全く思いつかなかった)

dpリストを作成し、これをupdateしていく。初期値は、dp=[nums[0]]。与えられた配列nums[1:]の要素を順々に見ていき、その要素がdpのどの要素よりも大きい場合には、それをdpの末尾にappendする。そうでない場合には、dpの中でそれ以上の値でindexが最も若い要素とそれを入れ替える。最終的に完成したlen(dp)が、求める値である。


```
着眼点としては、[1, 5, 3]が与えられた時、「３が加わる瞬間に５は用無し」になること。
注意点としては、「最終的に得られるdpリストの長さは、LSIの最大長に等しい」が、「それらの要素がLSIを作る要素を表しているわけではない」ということ。例えば、[1,3,5,2]が与えられた時、dpは、[1]>[1,3]>[1,3,5]>[2,3,5]となるが実際にLSIを作るのは[1,3,5]。
```

なお、「dpの中でそれ以上の値の一つ目の要素」を発見する過程でbinary searchを用いることでO(NlogN)時間になる。



### 所感

解法２が天才的。どうやったら思いつくんだ、、、
bisect_leftを使ったが、特に場合分けが要らなかった。これは、bisect_left(list, x)がlen(list)を返すケースを考えなくて良いから。


## 75. [Sort Colors](https://leetcode.com/problems/sort-colors/)

`Dutch national flag problem`   
0,1,2のいずれかの要素からなる配列(e.g. [1,2,2,1,0])を0, 1, 2のセクションに分かれるように、inplaceに入れ替えろ。

### 解法1 (my solution)

- １巡目に、0,1,2のそれぞれの出現数cnt0, cnt1, cnt2を数える。
- ２巡目に、(left, right) = (0, cnt1)から初めて、先頭のcnt0個の要素が0になるように(1or2 <-> 0)のswapを繰り返す。
- ３巡目に、(left, right) = (cnt0, cnt0+cnt1)から初めて、真ん中のcnt1個の要素が1になるように(1 <-> 2)のswapを繰り返す。


### 解法2 (似たようなの思いついたけど、edge caseをがややこそうだったので撤退)

次に、0, 2を置かれるべきindexをそれぞれ(p0, p1) = (0, len(nums)-1)に、またcurr = 0を定義する。


```python
while curr <= p1:   # =が必要らしい
    if curr == 0:
        (p0とcurrをswap)
        p0 += 1
        curr += 1       # currも右に動かすらしい。これは、currがp0よりも後ろにいるのを防ぐため??
    elif curr == 1:
        curr += 1
        # p1とのswap後には、currの箇所に２が来るかもしれないからまだcurrを右に動かすことはできない
    else:
        (p1とcurrをswap)
```

### 所感

解法１ではエッジケースを考え漏れていた。(cnt0=len(nums)つまり0のみの配列の場合や、cnt0+cnt1=len(nums)つまり0と1のみの配列の場合)
解法２は意外にもエッジケースを考える必要はなかった。ただし、コメントに書いた箇所の考慮がまたしっくりきていない。
